In [43]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
!unzip uncased_L-24_H-1024_A-16.zip
!nohup bert-serving-start -model_dir=./uncased_L-24_H-1024_A-16 > out.file 2>&1 &

from bert_serving.client import BertClient
bc = BertClient()



Requirement already up-to-date: bert-serving-server[http] in /usr/local/lib/python3.7/dist-packages (1.10.0)
--2021-03-22 10:21:21--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.195.128, 2607:f8b0:400e:c07::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1247797031 (1.2G) [application/zip]
Saving to: ‘uncased_L-24_H-1024_A-16.zip’

uncased_L-24_H-1024 100%[===================>]   1.16G  79.7MB/s    in 16s     

2021-03-22 10:21:36 (76.8 MB/s) - ‘uncased_L-24_H-1024_A-16.zip’ saved [1247797031/1247797031]

Archive:  uncased_L-24_H-1024_A-16.zip
   creating: uncased_L-24_H-1024_A-16/
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-24_H-1024_A-16/voca

In [50]:
import copy
import sys
import gensim
import pandas as pd
import sklearn
import numpy as np
from nltk.tokenize import word_tokenize
from bert_serving.client import BertClient
import re
import nltk
import os
nltk.download('punkt')
path = "/content/drive/My Drive"
os.chdir(path)
os.listdir(path)
def dealwithquery(filepath):
    f = open(filepath)
    line = f.readline()
    query={}
    j=0
    while line:
       thisquery=[]
    
       if ('<title>' in line):
         j=j+1 
         line=line.strip('<title> ')
         line=line.strip(' </title>\n')
         thisquery.append(line)
         query[j]=thisquery
       else:
         line = f.readline()
    return query
def getindex():
    Indexing = {}
    file=open("Trec_microblog11.txt",encoding='utf-8')
    line=file.readline()
    num=0
    array=[]
    while line:
        words=word_tokenize(line)
        line=line.strip(words[0])
        line=(remove_urls(line))
        if(line.isspace()):
          line = file.readline()
          continue
        Indexing[num]=words[0]
        array.append(line)
        line = file.readline()
        num=num+1

    length=len(Indexing)
    
    return (Indexing,length,array)
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)
def reclean(path):
  Indexing = {}
  f2 = open(path,"r")
  lines = f2.readlines()
  for line3 in lines:
    a=line3.split(' ')[0]
    if (a in Indexing):
      b=Indexing[a]
      b.append(line3.split(' ')[2])
      Indexing[a]=b  
    else:
      Indexing[a]=[line3.split(' ')[0]]     
  return Indexing
def main():
    qnum=0
    query=dealwithquery('topics_MB1-49.txt')
    (id,length,array)=getindex()
    sentences_vec = bc.encode(array)
    with open('Bert_new_output_ranking_based_on_A1.txt','w') as fa: 
      with open('Bert_new_output_ranking_A2.txt','w') as f: 
        for o in range(len(query)):
          qnum=qnum+1
          test_vec = bc.encode(query[o+1])
          score = np.sum(test_vec * sentences_vec, axis=1) / np.linalg.norm(sentences_vec, axis=1)
          topk_idx = np.argsort(score)[::-1][:length]
          rank=0
          count=0
          a=reclean('output.txt')
          b=list(a.values())[qnum-1]
          c=copy.deepcopy(b)
          for i in topk_idx:
              if(count<1000):
                count=count+1
                output1=str(qnum)+" Q0 "+str(id[i])+" "+str(count)+" "+str(score[i])+" query "+str(qnum)+" N0."+str(count) +"\n"
                print(output1)
                f.write(output1)
              if(id[i] in b):
                b.remove(id[i])
                oldindex=c.index(id[i])
                rank=rank+1
                output=str(qnum)+" Q0 "+str(id[i])+" "+str(rank)+" "+str(score[i])+" query "+str(qnum)+" N0."+str(rank) +" old Ranking is "+str(oldindex+1)+"\n"
                fa.write(output)

          
      f.close()       
    fa.close()
   

    

  
main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


流式输出内容被截断，只能显示最后 5000 行内容。
47 Q0 30430320247447553 501 8.590277 query 47 N0.501

47 Q0 29967642185965568 502 8.589468 query 47 N0.502

47 Q0 29977830074884096 503 8.589468 query 47 N0.503

47 Q0 29745173957181440 504 8.589468 query 47 N0.504

47 Q0 30122530492645376 505 8.589468 query 47 N0.505

47 Q0 31007696706605056 506 8.589468 query 47 N0.506

47 Q0 29786933068570624 507 8.589468 query 47 N0.507

47 Q0 30343746159640576 508 8.588913 query 47 N0.508

47 Q0 32202827560394754 509 8.588894 query 47 N0.509

47 Q0 34071899881345024 510 8.588693 query 47 N0.510

47 Q0 32497646199373824 511 8.588623 query 47 N0.511

47 Q0 30886022095048705 512 8.587338 query 47 N0.512

47 Q0 33507153557852160 513 8.586992 query 47 N0.513

47 Q0 29342782757797888 514 8.586482 query 47 N0.514

47 Q0 29356710409801730 515 8.586335 query 47 N0.515

47 Q0 30186641377198081 516 8.586307 query 47 N0.516

47 Q0 34025889058529280 517 8.585047 query 47 N0.517

47 Q0 33400885027540992 518 8.5849085 query 47 N0.518

